This code uses two different packages to extract names from a bunch of notes files and parses them into first, middle, and last names. To add gender, use Gender.Rmd file in github. 

In [ ]:
import en_core_web_sm
nlp = en_core_web_sm.load()

from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
st = StanfordNERTagger('/Users/seb2244/Downloads/stanford-ner-4.0.0/classifiers/english.all.3class.distsim.crf.ser.gz',
               '/Users/seb2244/Downloads/stanford-ner-4.0.0/stanford-ner.jar') 

from itertools import groupby

In [ ]:
import pandas as pd
import numpy as np
import docx2txt # for reading docx files
import os
import re

In [ ]:
# loop through folder of files and extract a row of names for each one

data = pd.DataFrame(columns = ["file_name", "name"])
file_list = []

for file in os.listdir(os.getcwd()):
    # this specific file is giving me problems for some reason 
    if file == "~$ompson, Joseph 43 USCI NARA.docx":
        continue
                
    # don't read txt files that are duplicates of existing docx files 
    elif (file.endswith(".txt")) & (file[:-4] not in file_list):
        file_list.append(file[:-4])
        file_object = open(file)
        text = file_object.read()
    
    # don't read docx files that are duplicates of existing txt files 
    elif (file.endswith(".docx")) & (file[:-5] not in file_list):
        file_list.append(file[:-5])
        text = docx2txt.process(file)  
    
    # will have to do few files of other types (doc, rtf) manually 
    else: 
        continue
        
    names = []
        
    # first method (using en_core_web_sm) 
    doc = nlp(text)
    for X in doc.ents:
        if X.label_ == 'PERSON':
            string = str(X)
            # get rid of special characters
            string = string.replace("/n", "").replace("/t", "").strip()
            string = re.sub(r"[^a-zA-Z\-., ]+", "", string)
            # restrict to a reasonable # of words for a name
            length = len(string.split())
            if (length > 1) & (length < 5):
                names.append(string)
                

    # second way (using stanford ner tagger)
    r=st.tag(text.split())
    for tag, chunk in groupby(r, lambda x:x[1]):
        if tag == "PERSON":
            name = " ".join(w for w, t in chunk)
            for n in name.split(";"):
                n = n.replace("/n", "").replace("/t", "").strip()
                n = re.sub(r"[^a-zA-Z\-., ]+", "", n)
                length = len(n.split())
                if (length > 1) & (length < 5):
                    names.append(n)

    names = list(set(names))
    for name in names:
        data = data.append({'file_name': file, 'name': name}, ignore_index=True)

In [ ]:
for i in range(len(data.index)):
    name = str(data.loc[i, 'name'])
    
    # take out hyphens at the end that were messing things up
    name = re.sub("[-]+$", "", name)
    # take out abbreviations at the end that represent place or company 
    if pd.isna(name):
        pass
    else:
        if re.match("[A-Za-z ]+ [A-Z]{2,}$", name):
            # check to make sure it doesn't end with our prespecified things
            if re.search("SR|JR|PHD|MD", name):
                pass
            else:
                name = re.sub("[ A-Z]{2,}$", "", name)
                data.loc[i, "name"] = name
    
    # for one word names, this algorithm doesn't really do anything bc it can't tell if it's a first or last name
    # so I'm just going to leave those columns as blank
    if len(name.split()) == 1:
        continue
        
    # use nameparser package to extract rest of names
    name = HumanName(name)
    data.loc[i, 'title'] = re.sub("\.", "", name.title)
    data.loc[i, 'first_name'] = re.sub("\.", "", name.first)
    data.loc[i, 'middle_name'] = re.sub("\.", "", name.middle)
    data.loc[i, 'last_name'] = re.sub("\.", "", name.last)
    data.loc[i, 'suffix'] = re.sub("\.", "", name.suffix)